# Real NVP

Paper: https://arxiv.org/pdf/1605.08803.pdf
Insight blog posts: https://blog.evjang.com/2018/01/nf1.html , https://blog.evjang.com/2018/01/nf2.html

## First post: distributions

Idea: make sampling straightforward by using bijections on top of a simple distribution.

It's easy to sample from a Gaussian and transform that sample to model an arbitrary distribution (latent -> sample). If that transformation is a bijection, it becomes straightforward to go from sample to latent as well (sample -> latent). One class of transformation which is a bijection is the concatenation of an affine transformation (i.e. in 2d, translate + scale + rotate + shear) and an invertible transform (like the leaky RelU0. The concatenation of many bijections is also a bijection, so they can be stacked.

So if we wanted to maximize the likelihood of a set of samples $y$ with respect to a bijection $f$, we would have:

$$\log p(y|f) = \log p(f^{-1}(y)) + \log \det J(f^{-1}(y))$$

The first part of this equation measures the likelihood of the data in the latent space. In other words, you take the observed data point, and you map it backwards into the latent space, where the evaluation of the likelihood is presumed to be easy (it certainly is for Gaussians). The second part is a correction for the size of local volume elements in the measurement vs. the latent spaces. 

To sample, sample from the latent distribution (e.g., a normal distribution), then project forward number of times. To evaluate the likelihood, project backward from the observation, measure the likelihood there, taking into account local changes in volume.

See this notebook:

https://github.com/ericjang/normalizing-flows-tutorial/blob/master/nf_part1_intro.ipynb

Note: this doesn't work very well out of the box.

## Second post: making NVP practical

Insight: autoregressive models can be made into normalizing flows -> they take Gaussians and modify them in a potentially bijective (reversible) way

Example: masked autoregressive flow. Here $x_i$ depends on previous $x$ as well as latent variables $u_i$. The transformation from $u_i$ is set up as a bijection, so that we can evaluate the probability of the data easily. However, sampling is slow because it depends on the previous $x_{i}$, and so is sequential.

Inverse autoregressive flow: similarly, but with dependencies between latent variables $u_i$ instead (different trade-off: learning slow, inference fast).

Parallel Wavenet: combine MAF and IAF to get fast training and inference.

## RealNVP

At each layer, partition variables into two subspaces; only allow some dependencies to keep the mapping bijective. Alternate the partitioning of subspaces (checkerboard masks on images). Allows fast inference and sampling. Works somewhat less well than MAF and IAF.